In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import re
import csv

def crawl_page(url=''):
    page= urllib.request.urlopen(url)
    soup_page= BeautifulSoup(page,'html.parser')
    return soup_page

def write_csv(line=[], file=''):
    with open(file, 'a', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=' ',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(line)

In [ ]:
class DataCollect():
    def __init__(self):
        self.__url_to_open='https://www.property24.com'
        self.__soup=crawl_page(self.__url_to_open)
        self.__area_url=[]
        self.__list_url=[]
        self.__property_url=[]
    
    def collect_all_first_page_url(self):
        other_links= self.__soup.find('div',class_='p24_popular').find(class_='col-8').find_all('a',class_='p24_bold')
        for link in other_links:
            self.__area_url.append(self.__url_to_open+link.get('href'))
        first_element_tag= self.__soup.find('div',class_='p24_popular').find(class_='col-8').find_all(class_='row')[1]
        first_link=first_element_tag.find('a').get('href')
        temp_soup= crawl_page(self.__url_to_open+first_link)
        parent_link= temp_soup.find(id='breadCrumbContainer').find_all('a')[1]
        self.__area_url.append(self.__url_to_open+parent_link.get('href'))
    
    def collect_all_url(self):
        self.collect_all_first_page_url()
        for link in self.__area_url:
            soup= crawl_page(link)
            link_house_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("houses-for-sale")).get('href')
            link_apart_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("apartments-for-sale")).get('href')
            link_townhouse_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("townhouses-for-sale")).get('href')
            self.__list_url.append(link_house_tag)
            self.__list_url.append(link_apart_tag)
            self.__list_url.append(link_townhouse_tag)

    def get_property_data(self, link=''):
        prop_soup= crawl_page(link)
        


    
    def get_property_url(self):
        self.collect_all_url()
        for link in self.__list_url:
            soup= crawl_page(self.__url_to_open+link)
            pagination= soup.find('div',class_='p24_pager').find_all('li')[-1].find('a').get('data-pagenumber')
            page_number= int(pagination)
            cur_soup=soup
            number_properties=0
            print(link)
            for page in range(1, page_number+1):
                print(page)
                properties= cur_soup.find_all('div',class_='js_resultTile')
                for prop_index in range(len(properties)-1):
                    prop_link= properties[prop_index].find('a').get('href')
                    write_csv([prop_link],'properties_url.csv')
                number_properties+= len(properties)
                next_button=cur_soup.find('div',class_='p24_pager').find('a', class_='pull-right').get('href')
                print(next_button)
                cur_soup= crawl_page(next_button)
            print(number_properties)



            

In [ ]:
dc= DataCollect()
dc.get_property_url()